In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import json

In [19]:
regexes = {"servings_per_container" : "(?<=\n)(.*)(?=Servings per container)", 
          "serving_size" : "(?<=Serving Size\n)(.*)(?=\n)",
          "calories" : "(?<=Calories\n)(.*)(?=\n)",
          "total_fat" : "(?<=Total Fat)(.*)(?=\n)",
          "saturated_fat" : "(?<=Saturated Fat)(.*)(?=\n)",
          "trans_fat" : "(?<=Trans Fat)(.*)(?=\n)",
          "chloestrol" : "(?<=Cholesterol)(.*)(?=\n)",
          "sodium" : "(?<=Sodium)(.*)(?=\n)",
          "carb" : "(?<=Carbohydrate)(.*)(?=\n)",
          "fiber" : "(?<=Fiber)(.*)(?=\n)",
          "sugar" : "(?<=Sugars)(.*)(?=\n)",
          "protein" : "(?<=Protein)(.*)(?=\n)",
          "ingredients" : "(?<=Ingredients:)(.*)(?=\n)",
          "allergy" : "(?<=Contains:)(.*)(?=\n)"}

In [20]:
def process_subsection(driver):
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@title = 'Open the nutrition label for this item']"))
    )
    food_items = driver.find_elements(By.XPATH, "//*[@title = 'Open the nutrition label for this item']")
    nutrition_infos = []
    for food_item in food_items:
        driver.execute_script("arguments[0].scrollIntoView(false);", food_item)
        try:
            food_item.click()
        except:
            continue
        print(food_item.text)
        label = WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.CLASS_NAME, 'horizontal-flex-container'), food_item.text)
        )
        label = driver.find_element(By.CLASS_NAME, 'horizontal-flex-container')
        try:
            nutrition_infos.append(label.get_attribute("innerText"))
        except:
            continue
    master_list = {}
    for food_item in nutrition_infos:
        info = {}
        for item in regexes:
            regex = re.compile(regexes[item])
            try:
                text = regex.findall(food_item + "\n")[0].replace(u'\xa0', u' ').strip()
                info[item] = text
            except:
                print("skipped " + item + " for " +  food_item.split("\n")[0])
                continue
        master_list[food_item.split("\n")[0]] = info
    return master_list

In [21]:
def process_all_times(driver):
    subsections = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@role = 'button']"))
    )
    subsections = driver.find_elements(By.XPATH, "//*[@role = 'button']")
    all_subsection = {}
    for subsection in subsections:
        driver.execute_script("arguments[0].scrollIntoView(false);", subsection)
        subsection.click()
        time.sleep(0.5)
        master_list = process_subsection(driver)
        all_subsection[subsection.get_attribute("innerText")] = master_list
        time.sleep(0.5)
        try:
            driver.execute_script("window.scrollTo(0, 220)") 
            time.sleep(0.5)
            subsection.click()
        except:
            print("skipping click for " + subsection.get_attribute("innerText"))
    return all_subsection

In [22]:
def process_restaurant(driver, restuarant_name, link):
    restaurant = {}
    try:
        time_of_day = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "cbo_nn_menuLink"))
        )
        time_of_days = driver.find_elements(By.CLASS_NAME, "cbo_nn_menuLink")
    except:
        time_of_days = []
    if len(time_of_days) != 0:
        i = 0
        for time_of_day in time_of_days:
            temp = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "cbo_nn_menuLink"))
            )
            temp = driver.find_elements(By.CLASS_NAME, "cbo_nn_menuLink")
            time.sleep(0.5)
            name_of_menu = temp[i].get_attribute("innerText")
            if (name_of_menu in restaurant):
                i += 1
                continue
            driver.execute_script("arguments[0].scrollIntoView(false);", temp[i])
            temp[i].click()
            all_subsection = process_all_times(driver)
            restaurant[name_of_menu] = all_subsection
            time.sleep(1)
            
            driver.execute_script("window.scrollTo(0, 220)") 
            time.sleep(1)
            link = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Back"))
            )
            link.click()
            time.sleep(0.5)
            i+=1
    else:
        all_subsection = process_all_times(driver)
        restaurant["all_items"] = all_subsection
        time.sleep(1)
    return restaurant

In [24]:
driver = webdriver.Chrome()
driver.get("https://netnutrition.cbord.com/nn-prod/duke")

restaurants = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "col-12"))
)

restaurants = driver.find_elements(By.CLASS_NAME, "col-12")
master = {}
#iterate over all restaurants

for i in range (0, len(restaurants)):
    restaurant = restaurants[i]
    restuarant_name = restaurant.get_attribute("innerText")
    print(restuarant_name)
    link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, restuarant_name))
    )
    driver.execute_script("arguments[0].scrollIntoView(false);", link)
    
    link.click()
    time.sleep(.5)
    ret = process_restaurant(driver, restuarant_name, link)
    master[restuarant_name] = ret
    #save intermediate results so we can start after a crash
    f = open("temp" + str(i) + ".json", "w")
    json.dump(master, f)
    f.close()
    
    f = open("temp_single" + str(i) + ".json", "w")
    json.dump(ret, f)
    f.close()
f = open("final.json", "w")
json.dump(master, f)
f.close()


Bella Union
Americano Decaffeinated
Americano Regular
Cafe Au Lait Almond Milk
Cafe Au Lait Oat Milk
Cafe Au Lait Skim Milk
Cafe Au Lait Soy Milk
Cafe Au Lait Whole Milk
Cappuccino Almond Milk
Cappuccino Oat Milk
Cappuccino Skim Milk


KeyboardInterrupt: 